# Neural Style Transfer!

In [1]:
# Load Dependencies
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras.backend as K

from keras.applications import vgg19
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg19 import preprocess_input

from keras.models import Model

from helpers.image import get_image

# Setup Config Variables / Constants

In [9]:
# Config Variables / Constants

# Images
TARGET_HEIGHT = 512
TARGET_WIDTH = 512
TARGET_CHANNELS = 3

# Image Paths
CONTENT_PATH = '../data/turtle.png'
STYLE_PATH = '../data/van-gogh.jpg'

# VGG 19 Layers
CONTENT_LAYERS = ['block5_conv2']
STYLE_LAYERS = [('block1_conv1', 0.2),
                ('block2_conv1', 0.2),
                ('block3_conv1', 0.2),
                ('block4_conv1', 0.2),
                ('block5_conv1', 0.2)]

print('Num. Content Layers:', len(CONTENT_LAYERS))
print('Num. Style Layers:  ', len(STYLE_LAYERS))

Num. Content Layers: 1
Num. Style Layers:   5


# Helper Functions

### Image Helper Functions

In [4]:
# Image Helper Functions
def show_img(image, title=''):
    plt.imshow(image)
    if title:
        plt.title(title)
    plt.show()

def vgg19_preprocess_img(img):
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def preprocess_img(path_to_img, show = False, title = ''):
    img = load_img(path_to_img, target_size=(TARGET_HEIGHT, TARGET_WIDTH))
    if show:
        show_img(img, title)
    img = img_to_array(img)
    return vgg19_preprocess_img(img)

def deprocess_img(img):
    x = img.copy()
    
    if len(x.shape) == 4:
        x = np.squeeze(x, 0)
    assert len(x.shape) == 3, ("Input to deprocess image must be an image of "
                             "dimension [1, height, width, channel] or [height, width, channel]")
    if len(x.shape) != 3:
        raise ValueError("Invalid input to deprocess_img()")
    
    # Inverse of preprocessing
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    
    # Ensure we're in the range of (0, 255)
    x = np.clip(x, 0., 255.).astype('uint8')
    return x

def generate_img(show = False):
    img = np.random.randn(TARGET_HEIGHT, TARGET_WIDTH, TARGET_CHANNELS)
    img *= 255.0
    img = np.clip(img, 0, 255)
    if show:
        show_img(img, 'Generated Image')
    return vgg19_preprocess_img(img)

### Loss Helper Functions

In [5]:
# Loss Helper Functions
def mean_squared_error(a, b):
    return tf.reduce_mean(tf.square(a - b))

def gram_matrix(a):
    return tf.matmul(tf.transpose(a))

In [23]:
# Model Helper Functions
def get_model():
    """ Load VGG19 model with the appropriate outputs """
    model = vgg19.VGG19(include_top=False, weights='imagenet')
    
    content_output = [model.get_layer(layer_name).output for layer_name in CONTENT_LAYERS]
    style_output = [model.get_layer(layer_name).output for layer_name,coeff in STYLE_LAYERS]
    model_outputs = content_output + style_output
    
    return Model(inputs=model.inputs, outputs=model_outputs)

## Load Content, Style, and Generated Images

In [30]:
# Load Content, Style, and Generated Images
content_img = preprocess_img(CONTENT_PATH, show=False, title='Content Image')
style_img = preprocess_img(STYLE_PATH, show=False, title='Style Image')
generated_img = generate_img(show=False)

print('Content Shape:  ', content_img.shape)
print('Style Shape:    ', style_img.shape)
print('Generated Shape:', generated_img.shape)

Content Shape:   (1, 512, 512, 3)
Style Shape:     (1, 512, 512, 3)
Generated Shape: (1, 512, 512, 3)


# Questions to Answer Later
1. What does gram matrix do?
2. What does mean squared error do?
3. 